# 📚 Импорт библиотек

In [1]:
import pandas as pd
import os

from copy import deepcopy

# 👛 Импорт получателей дропа

In [2]:
# Путь к папке с CSV
folder_path = "drop_getters"

# Получаем список всех файлов .csv в папке
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Считываем и объединяем все файлы
df_list = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

# Объединяем в один DataFrame
drop_getters_df = pd.concat(df_list, ignore_index=True)
drop_getters_df.sample(7)

,evt_block_time,user,airdropped_tokens,evt_tx_hash
199990,2024-09-22 12:42:19.000 UTC,0xb46c8a56a33890587e5f88f563ee4ffc5f595ce5,16.032740,0x37f23ef6b2fbf632661dd8f10bffd4260d988dcbe7ff...
11028,2024-06-21 08:43:58.000 UTC,0xcfc6f5b149ffd802139d54a4e80af31dce605d8a,6.487248,0xd034421994059c5088a2b427acd8d69810d36326fa63...
318999,2024-06-29 06:44:41.000 UTC,0xaa57e277df6c6b24c6fc55f27c5547dd5998a8e6,20.815543,0x4a2c3dc41aa20be7683558dbd0cb35c4222965bea3e6...
692006,2024-06-25 14:15:47.000 UTC,0xe3b89e352aadd558489ec87f6b64b299c0c4b553,188.242775,0x1ecf984de07b5dd90c6c25dc5b643a646d09bdcd082a...
813268,2024-06-20 23:23:53.000 UTC,0x9adcabc8dd12d9f522598ce28b4404d13ad58e91,7.342970,0xd8567d2f88b17a78c903992f4460bda66605a1781445...
162619,2024-06-20 11:37:11.000 UTC,0x88731ad013cd409dfcf49bd5f103ab040066eff4,359.455020,0x6df866979f9c508278772871292aaba7690fc5623ba6...
111720,2024-06-20 11:16:43.000 UTC,0x5e3283b503c9954fe515fca0684c430c30b04c9a,112.746928,0x86b87be1b3f54a00db9eee654766444cb72e50a84aab...


In [3]:
drop_getters_df.dtypes

evt_block_time        object
user                  object
airdropped_tokens    float64
evt_tx_hash           object
dtype: object

In [4]:
print(drop_getters_df.shape)
print(len(drop_getters_df["user"].unique()))
print(len(drop_getters_df["evt_tx_hash"].unique()))
print(drop_getters_df["evt_block_time"].max())
print(drop_getters_df["evt_block_time"].min())

(831850, 4)
737292
831850
2024-11-29 19:31:31.000 UTC
2024-06-20 10:39:07.000 UTC


In [5]:
dg_df = deepcopy(drop_getters_df)
dg_df["evt_block_time"] = pd.to_datetime(dg_df["evt_block_time"])
dg_df = dg_df[
    dg_df["evt_block_time"] <= pd.Timestamp("2024-06-27", tz="UTC")
]  # чтобы убрать слишком поздние получки
dg_df = dg_df[["user", "airdropped_tokens"]]
dg_df = dg_df.groupby("user").sum().reset_index()
dg_df["get_drop"] = 1
dg_df

,user,airdropped_tokens,get_drop
0,0x00000000091b2041a94d32b05556c52028161b28,68.786506,1
1,0x000000000a38444e0a6e37d3b630d7e855a7cb13,773.568119,1
2,0x000000000bb6a011db294ce3f3423f00eac4959e,113.580528,1
3,0x000000002db9eae8bd9cc134752500f2c38334f3,1196.430469,1
4,0x000000003af6916de52d3425b4e49d7f97edebee,14.593708,1
...,...,...,...
593256,0xffffebe5327691f27ea1df5e427b88312f124929,67.404531,1
593257,0xfffff6c029985fae0a82725ea77e011c47c9a65d,138.349661,1
593258,0xfffff9b1c2c387b1e3d19af292d91d913374f42b,115.578298,1
593259,0xffffffd7cc631a49dc9dc7a14ef83875360e2fee,256.142153,1


# 🧑‍🌾 Импорт опубликованных sybil-адресов

In [6]:
with open("sybil_list_combined.csv", "w") as outfile:
    for fname in [
        "sybil_adresses/layerzero-main/fcfs_provisional_list.csv.part1",
        "sybil_adresses/layerzero-main/fcfs_provisional_list.csv.part2",
        "sybil_adresses/layerzero-main/fcfs_provisional_list.csv.part3",
    ]:
        with open(fname) as infile:
            outfile.write(infile.read())

In [7]:
s_df = pd.read_csv("sybil_list_combined.csv")
s_df["user"], s_df["airdropped_tokens"] = s_df["Address"], s_df["Allocation"]
s_df = s_df[["user", "airdropped_tokens"]]
s_df = s_df.groupby("user").sum().reset_index()
s_df["get_drop"] = 0
s_df

,user,airdropped_tokens,get_drop
0,0x00000000037a08ce8c4917a5c58f40c85367df0c,6.457637,0
1,0x0000000a362b7ae7239e186b2bbeb943305c59ad,1.164103,0
2,0x0000007da5480322da3961f499c50c4f438ac76b,2.406804,0
3,0x0000411364ffbf912961a4640d8c1b1804dd7a94,19.479719,0
4,0x000043055c40cda965b053523e17888649ad42df,2.491000,0
...,...,...,...
151967,0xfffd6e0d6e85bf66791b985ab1a070fb82761ec9,4.482010,0
151968,0xfffd7a81582ff9622575eb09f245e33593b34352,1.011320,0
151969,0xfffd800286ffc6581a01fcbb237df7573c4f3a5a,0.511255,0
151970,0xfffef4587dad2641b307234a1c4429f47a4c16a4,2.969670,0


# 🤖 Соединяем в один датасет

In [8]:
df_final = (
    pd.concat([dg_df, s_df])
    .reset_index(drop=True)
    .sample(frac=1, random_state=22)
    .reset_index(drop=True)
    .reset_index(drop=False)
)
df_final

,index,user,airdropped_tokens,get_drop
0,0,0x934204e4617792b8ec2a34f0330a19ab3393041f,5.464521,0
1,1,0xa99979dca47c7c41262936521d59d9a922f7a1d2,4.161325,0
2,2,0x5fc7b11e3281b7404573465bc99d9aebc87ec59b,5.756350,0
3,3,0x7c56d095261c0c9921adf95212f499018340bdde,19.357885,1
4,4,0xa33f601b04ba846586266bfae01ae47d5362eff4,6.113811,1
...,...,...,...,...
745228,745228,0x5e73cf9cebd82d8c192f538f41186b55965e6fc6,25.185039,1
745229,745229,0x335b7c6633a29e43dbe2f7804803cc8243f2a830,65.942124,1
745230,745230,0xecebbda750e02674eab8929c69e9de5b30c4446f,26.084444,1
745231,745231,0x45d1d1124a10dafdf0fe6c6e12a1dea9c1c6db25,159.581385,1


In [9]:
df_final.to_pickle("df_addresses.pickle")